# Kaggle Competition Homework Set  
## By Jacob Tiede  
### Exploritory Data Analysis:  
**Note: all of this analysis will submitted in a separate file since I used R to do the EDA. This will just be a summary of the findings**  

Looking through the data the things that jumped out to me were class imbalances. One can see that males are much more represented in the data than females, people who earn less than 50,000$ a year are much more prevalent than those that earn more, and people who identified as white were more prevalent than any other race. After trying multiple techniques (oversampling to make the response variable balanced, and undersampling, as well as doing bagged models where the models were trained on balanced data) I've found that the only class imbalance that helps with predictive strenth is accounting for the sex of participants. I've also decided to take the log of age and hours per week (despite the fact that my methods should be somewhat resilient to ourliers), because this seems to help the accuracy a little. Other than that the EDA showed some possible feature crosses to try, and the fact that there are many outliers (also, I did observe these class imbalances in the testing set). 

### Feature Engineering: 
I decided to include three feature crosses between education and gender, education and race, and gender and occupation. After trying many crosses these ended up being the two that helped predictions the most though more were possible (and the exploritory data analysis may have suggested others, but these were the only ones that helped). I opted to drop the education column since it seemed to be redundant to include it with the education number (as indicated by the EDA). I've also decided to include a 'net capital' column  which will just be the capital gain - capital loss (since this is probably more important than either number in isolation), and a weighted buying power column (hours per week multiplied by education number) since people who work more with less education might still earn more than people with high education who work very few hours, so I thought I would tell the model this explicitly.

### Methods:  
I opted for a tree-based model because, after doing some research, I found that neural networks/deep learning are only effective for a very large amount of data (way more than what we were given), and I wanted to do an ensemble method that would converge somewhat quickly, so trees seemed like a natural choice. Also there were many outliers in the data and I believe that tree methods (if tuned correctly) are more robust to this problem than other methods. To solve the class imbalance problems I created a method that would randomly generate samples that contained an even number of male and female participants, I then trained both an XGBoosted forest and a Catboosted forest for each sample (each of the libraries use different methods of splitting the data so using both is better than using either separately). I then ensemble these together by finding each one's accuracy on a validation set, and I weight each one according to $weight = \frac{1}{1-accuracy}$. I chose this weight function since it exaggerates small differences in accuracy of each model (since all of them have accuracies that are close to one another), but it also gives all models some relatively high base-line importance (unlike a function like $-log(1-x)$). The prediction is then:
$$
\sum_{i=1}^{N}(weight(i)*prediction(i))
$$
Where prediction(i) = 1 if the ith model predicts 1 and prediction(i) = -1 if the ith model predicts 0. Then, if that sum is > 0 then our model would predict a class of 1, and if it is < 0 we would predict a 0. Then, as some side notes I used a label encoder rather than one hot encoding becuase label encoding generates no new features so it is much faster in general (especially when I am making 150 models to ensemble), and I opted to use yeo-johnson normalization with scaling (which this particular scalar does by default unless you tell it not to), becuase it got the best results of all the scalar/normalization combinations that I tried.

In [2]:
#Per the Syllabus see citations for all these packages at the end of the document
import xgboost as xgb
import numpy as np
import pandas as pd
import catboost as cat
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
"""
Description: Due to over representation of people earning less than 50K per year and men in the data I have to
make this method to generate N sample with numPerSample data points that enforce one equal numbers of these 
two conditions
Note: I currently do not use the equal representation of 50K per year, but I left it in since I might use it for
a later iteration

@param N: Number of samples to take with balanced classes
@param numPerSample: number of data points in each sample
@param trainData: The data that we will use to generate balanced classes of men and women
@param trainResponse: The data we will use to generate balanced classes of <50K and >50k earnings
@param genderindex: tells us which column of trainData contains information on gender

Return: an output matrix where we have a repeating pattern every 4 rows. The first row will be a sample of female
indicies, the second will be male indicies, the third will be a list of indicies where someone earned >=50k and the
fourth will be people will earn <=50k. This will generate a matrix that is 4N by numPerSample in size
"""
def baggingIndices(N, numPerSample, trainData, trainResponse, genderindex):
    #initialize our return matrix
    output = []
    #initialize a list that will contain all indicies that have a female in them (though this is arbitrary)
    listOfFemaleInd = []
    #Initialize a list of row indicies that have a male participant
    listOfMaleInd = []
    #extract the gender column from the data
    gender = trainData[:,genderIndex]
    #since we are assuming that the data has already been normalized we will set our discriminant as the first value
    #of gender (this is why calling the lists female and male are arbitrary since the first value could be either)
    disc = gender[0]
    for i in range(0, len(gender)):
        #If row i has our discriminant gender assume that it is female (again it doesn't matter since we are generating
        #equal numbers of both)
        if gender[i] == disc:
            listOfFemaleInd.append(i)
        #Else assume it they are male
        else:
            listOfMaleInd.append(i)
    #Do a similar process for people earning >=50k vs <=50k
    listOfPosOutcomes = []
    listOfNegOutcomes = []
    for i in range(0, trainResponse.shape[0]):
        if trainResponse[i] == 1:
            listOfPosOutcomes.append(i)
        else:
            listOfNegOutcomes.append(i)
    #Set seed for reproducability
    np.random.seed(0)
    #generate N samples that will be put in our output matrix
    for i in range(0, N):
        #if i==0 we need to use a stack method to generate the matrix because of the way numpy arrays work
        
        #Essentially for each sample we will generate a random sample of indicies from our list of indices for
        #each class that we want to balance. We will get numPerSample indicies from each of our lists without
        #replacement, so we will have ensured that our end list of indicies have the same number of either gender
        #or >=50k <=50k. These will then be stacked in the output matrix
        if i > 0:
            femaleInd = np.random.choice(listOfFemaleInd, numPerSample, replace = False)
            maleInd = np.random.choice(listOfMaleInd, numPerSample, replace = False)
            posInd = np.random.choice(listOfPosOutcomes, numPerSample, replace = False)
            negInd = np.random.choice(listOfNegOutcomes, numPerSample, replace = False)

            output = np.append(output, [femaleInd], axis = 0)
            output = np.append(output, [maleInd], axis = 0)
            output = np.append(output, [posInd], axis = 0)
            output = np.append(output, [negInd], axis = 0)
        else:
            femaleInd = np.random.choice(listOfFemaleInd, numPerSample, replace = False)
            maleInd = np.random.choice(listOfMaleInd, numPerSample, replace = False)
            posInd = np.random.choice(listOfPosOutcomes, numPerSample, replace = False)
            negInd = np.random.choice(listOfNegOutcomes, numPerSample, replace = False)

            output = femaleInd
            output = np.stack((output, maleInd))
            output = np.append(output, [posInd], axis = 0)
            output = np.append(output, [negInd], axis = 0)
    return output

"""
Description: Given a list of XGboosted, and Catboosted models each with a different accuracy on a validation set
this method will ensemble them to get one set of predictions. It does this by predicting what each model says on
the testData and weights that prediction according to 1/(1-accuracy of model i). 

@param baggedModelsXG: the XGBoosted models that were trained on data with balanced classes
@param confidXG: accuracy of each XGBoosted model on a validation set
@param baggedModelsCat: the catBoosted models that were trained on data with balanced classes
@param confidCat: accuracy of each catBoosted model on a validation set

return: a vector of what we classifed each entry in the test data
"""
def baggedPredict(baggedModelsXG, confidXG, baggedModelsCat, confidCat, testData):
    #make a dmatrix of the testData so we can call predict
    Dmatrix_test = xgb.DMatrix(testData, label=None)
    #Initialize a consensus matrix (this will keep track of what each model says about each data point)
    #each row will represent a model and the ith column will represent the jth model's prediction on the ith data
    #point (jth row)
    consensus = np.zeros((len(baggedModelsXG) + len(baggedModelsCat)+1, testData.shape[0]))
    #Iterate through each XG boosted model
    for i in range(0,len(baggedModelsXG)):
        #generate predictions
        predictions = baggedModelsXG[i].predict(Dmatrix_test)
        #Get the best prediction in each line
        mostconfidXGPred = np.zeros(predictions.shape[0])
        for j in range(0, predictions.shape[0]):
            if predictions[j] > .5:
                mostconfidXGPred[j] = 1
            else:
                #set this to -1 so that we don't have a bias toward positive cases
                mostconfidXGPred[j] = -1
            consensus[i,j] = 1/(1-confidXG[i])*mostconfidXGPred[j]
    #Iterate through each cat boosted model
    for i in range(len(baggedModelsXG), len(baggedModelsXG) + len(baggedModelsCat)):
        predictions = baggedModelsCat[i-len(baggedModelsXG)]. predict(testData)
        for j in range(0, len(predictions)):
            #remove bias toward positive cases
            if predictions[j] == 0:
                predictions[j] = -1
            consensus[i,j] = 1/(1-confidCat[i-len(baggedModelsXG)])*predictions[j]
    #each rowsum will represent a single prediction
    consensus = np.sum(consensus, axis = 0)
    #turn predictions either into a 0 or a 1
    for i in range(0, len(consensus)):
        if consensus[i] > 0:
            consensus[i] = 1
        else:
            consensus[i] = 0
    return consensus

#BEGIN PREPROCESSING
#TRAIN DATA LOADING, FEATURE CROSSES, NEW FEATURE CREATION, AND TRANSFORMS
#load the training data in with names given to each column
trainData = pd.read_csv("./Pr4Data/train-features.csv", header=None,
                       names = ["age", "workclass", "fnlwgt", "education", "education_num", 
                                "marital_status", "occupation", "relationship",
                                "race", "sex", "capital_gain", "capital_loss", "hoursPerWeek", "nativeCountry"])
#create new features
#NOTE: lambda is a make shift function that you define temporarily. So what this line of code means is:
#apply the function lambda to each row of the data along axis 1 (the columns)
trainData['net_capital'] = trainData.apply(lambda row: row.capital_gain - row.capital_loss, axis = 1) 
trainData['weightedBuyingPower'] = trainData.apply(lambda row: row.education_num * row.hoursPerWeek, axis = 1)
#take the log of age and hoursPerWeek
trainData['ageLog'] = trainData.apply(lambda row: np.log(row.age), axis = 1)
trainData['hoursPerWeekLog'] = trainData.apply(lambda row: np.log(row.hoursPerWeek), axis = 1)
trainData.drop(columns='age', inplace = True)
trainData.drop(columns='hoursPerWeek', inplace = True)
#Create crossed columns:
trainData['GenderOccupation'] = trainData.apply(lambda row: row.occupation + row.sex, axis = 1)
trainData['EducationGender'] = trainData.apply(lambda row: row.sex + row.education, axis = 1)
trainData['EducationRace'] = trainData.apply(lambda row: row.education + row.race, axis = 1)

#drop redundant column
trainData.drop(columns='education', inplace = True)


#TEST DATA LOADING, FEATURE CROSSES, NEW FEATURE CREATION, AND TRANSFORMS
#load test data and do the same things to it that we did to the train set
test = pd.read_csv("./Pr4Data/test-features.csv", header=None,
                       names = ["age", "workclass", "fnlwgt", "education", "education_num", 
                                "marital_status", "occupation", "relationship",
                                "race", "sex", "capital_gain", "capital_loss", "hoursPerWeek", "nativeCountry"])
#Everything done same as trainData
test['net_capital'] = test.apply(lambda row: row.capital_gain - row.capital_loss, axis = 1) 
test['weightedBuyingPower'] = test.apply(lambda row: row.education_num * row.hoursPerWeek, axis = 1) 
test['ageLog'] = test.apply(lambda row: np.log(row.age), axis = 1)
test['hoursPerWeekLog'] = test.apply(lambda row: np.log(row.hoursPerWeek), axis = 1)
test.drop(columns='age', inplace = True)
test.drop(columns='hoursPerWeek', inplace = True)
test['GenderOccupation'] = test.apply(lambda row: row.occupation + row.sex, axis = 1)
test['EducationGender'] = test.apply(lambda row: row.sex + row.education, axis = 1)
test['EducationRace'] = test.apply(lambda row: row.education + row.race, axis = 1)
test.drop(columns='education', inplace = True)

#aggregate the train and test data so that we can train our label encoder and our normalization properly
dataToTrainLabelEncoders = pd.concat([trainData,test], ignore_index=True)

#use label encoder on the categorical columns
listOfLabelEncoders = []
listOfColumns = list(dataToTrainLabelEncoders.columns)
l=0
#for each column we will train and fit our label encoder
for i in range(0, len(listOfColumns)):
    if isinstance(dataToTrainLabelEncoders[listOfColumns[i]][0], str):
        le = preprocessing.LabelEncoder()
        #Note: don't need to remove non categorical columns from consideration, because this 
        #method won't affect quant cols
        le.fit(dataToTrainLabelEncoders[listOfColumns[i]])
        listOfLabelEncoders.append(le)
        dataToTrainLabelEncoders[listOfColumns[i]] = listOfLabelEncoders[l].transform(
                                                        dataToTrainLabelEncoders[listOfColumns[i]])
        l+=1
        
#Use the label encoders trained above on the train data set so we can train our classifiers        
listOfColumns = list(trainData.columns)
l=0
for i in range(0, len(listOfColumns)):
    if isinstance(trainData[listOfColumns[i]][0], str):
        trainData[listOfColumns[i]] = listOfLabelEncoders[l].transform(trainData[listOfColumns[i]])
        l+=1

#record the column with sex to use in finding my bagging indices
trainingColumns = list(trainData.columns)
genderIndex = trainingColumns.index('sex')
#get trainResponse
trainResponse = pd.read_csv("./Pr4Data/train-output.csv", header=None)
trainResponse = trainResponse.values #make it into a np array

#Use a yeo-johnson transformer (the default of powerTransformer) to normalize and scale the data
#NOTE: Unless you tell power transformer not to it will also scale the data (as well as normalize)
scaler = preprocessing.PowerTransformer()
scaler.fit(dataToTrainLabelEncoders)
trainData = scaler.transform(trainData)

#Split the data into test and train, then split again so we can know when our model is done training
train_x, valid_x, train_y, valid_y = train_test_split(trainData, trainResponse, test_size=0.1,random_state=11)
train_x2, valid_x2, train_y2, valid_y2 = train_test_split(train_x, train_y, test_size=0.1,random_state = 15)

#create 150 models
N = 150 
#each model will account for 4000 data points in each class
numPerSample = 4000 
#get the indices that we will create our models with
bagging = baggingIndices(N,numPerSample, train_x2, train_y2, genderIndex)
#initialize lists where we will keep all the models generated
baggedModelsXG = []
ConfidModelsXG = []
baggedModelsCat = []
ConfidModelsCat = []
#params for XGBoost, fitted these basically through trail and error for the best results, though I understand that
#There are a few heuristics to use.

#Brief description of the paramters:
#eta can be thought of as the step size, .3 seems to be the best choice for my model
#max_depth is how many splits our tree can be, I set this to 6 (which is fairly deep to me) because I will use the
#XGBoosted trees as slightly overfit models, and the catboosted trees as slightly underfit models
#objective determines what XGBoost is trying to minimize, for binary classification use logistic
#num parrallel tree determines how many boosted trees we will include set to 15 because this gave good results
#lambda is a regularization parameter (think ridge regession)
#tree_method: algorithm to construct the trees, used exact because I wanted more accuracy (took a hit on 
#preformance though)
#max delta step: added this constraint to help with overfitting
#NOTE: per the XGBoost documentation a dictionary is used for parameters, which is why I used a dictionary
params = {
        'eta': 0.3, 
        'max_depth': 6,  
        'objective': 'binary:logistic',  
        'num_parallel_tree' : 15,
        'lambda': .1,
        'tree_method': 'exact',
        'max_delta_step' : 5}
NumberOfSteps = 60  # The number of training iterations

#for each set of 4 rows in my matrix of sampled rows (this is required because of the way I constructed the 
#bagging matrix)
for i in range(0, int(bagging.shape[0]/4)):

    #pick indicies that make the sexs represented in the data even
    #This j will give the set of female indicies with exactly 4000 entries
    j = i*4
    trainIndicesForModel = np.zeros(train_x2.shape[0], dtype = bool)
    trainIndicesForModel[bagging[j,:]] = True
    #Choose the other row with 4000 indicies (the one for male indicies)
    j+=1
    trainIndicesForModel[bagging[j,:]] = True
    dataToTrainIthModel = train_x2[trainIndicesForModel, :]
    #Subset our response
    responceToTrainIthModel = train_y2[trainIndicesForModel]
    
    #make dmatrices with these so we can train XGBoost
    Dmatrix_train = xgb.DMatrix(dataToTrainIthModel, label=responceToTrainIthModel)
    Dmatrix_test = xgb.DMatrix(valid_x2, label=None)
    
    
    #train the ith model
    clfXGB = xgb.train(params, Dmatrix_train, NumberOfSteps)
    #append the trained model to our list of models
    baggedModelsXG.append(clfXGB)

    #the next set of code will get our confidence in the ith xgboost model
    predictions = clfXGB.predict(Dmatrix_test)
    #Get the best prediction in each line
    mostConfidPred = np.zeros(predictions.shape[0])
    for j in range(0, predictions.shape[0]):
        if predictions[j] > .5:
            mostConfidPred[j] = 1
        else:
            mostConfidPred[j] = 0
    #Save this as our confidence in XGBoost:
    XGConfid = accuracy_score(valid_y2, mostConfidPred)
    #append our confidence in the ith model
    ConfidModelsXG.append(XGConfid)
    
    #Cat boosting:
    #learning rate can be thought of as step size
    #silent keeps it from printing a ton of information
    #Set max depth to 3 because I wanted this model to be slightly underfit
    #num_trees is kind of like the number of iterations of training, set this to 500 because it should converge
    #by then
    #reg_lambda: also a regularization
    #scale pos weight: this will try to account for the class imbalance in the response variable 
    clfCat = CatBoostClassifier(learning_rate = .6, silent = True, max_depth = 3, num_trees = 500, 
                                reg_lambda = .1, scale_pos_weight = 4/3)
    clfCat.fit(dataToTrainIthModel, responceToTrainIthModel)
    #Append the model to the list of cat boosted models
    baggedModelsCat.append(clfCat)
    #get the predictions
    predictions = clfCat.predict(valid_x2)
    #use the accuracy to find our confidence in the cat boosted models
    CatConfid = accuracy_score(valid_y2, predictions)
    ConfidModelsCat.append(CatConfid)


predictions = baggedPredict(baggedModelsXG, ConfidModelsXG, baggedModelsCat, ConfidModelsCat, valid_x)
#print our accuracy on the validation set
print("Accuracy on the test set:")
print(accuracy_score(valid_y, predictions))


Accuracy on the test set:
0.8854774332207553


Below is my code for getting the submission documents

In [3]:
#Open the submission file with writing capabilities
output = open("Submission.csv", 'w')
#Overwrite anything that was previously written in the document (start the cursor at position 0)
output.truncate(0)
#Formot the first row
output.write("Id,Category" + "\n")

#Do the same preprocessing on the test set that we did on the train
#Label encode
listOfColumns = list(test.columns)
l = 0
for i in range(0, len(listOfColumns)):
    if isinstance(test[listOfColumns[i]][0], str):
        test[listOfColumns[i]] = listOfLabelEncoders[l].transform(test[listOfColumns[i]])
        l+=1
#Normalize with the same scaler we used to normalize the train data
test = test.values
test = scaler.transform(test)
#Get our predictions
predictions = baggedPredict(baggedModelsXG, ConfidModelsXG, baggedModelsCat, ConfidModelsCat, test)
#Write all of the predictions in the format that Kaggle expects
for i in range(0, test.shape[0]):
    if predictions[i] <.5:
        output.write(str(i) + ',' + str(0) + "\n")
    else:
        output.write(str(i) + ',' + str(1) + "\n")
#Close the file
output.close()

Full citations:    
SciKitLearn:  
Pedregosa, F., Varoquaux, G., Gramfort, A., Michel, V., Thirion, B., Grisel, O., … Duchesnay, É. (1970, January 1). Scikit-learn: Machine Learning in Python. Retrieved from http://jmlr.csail.mit.edu/papers/v12/pedregosa11a.html  
Pandas:  
Mckinney, W. (2010). Data Structures for Statistical Computing in Python. Proceedings of the 9th Python in Science Conference. doi: 10.25080/majora-92bf1922-00a  
XGBoost:  
Chen, T., & Guestrin, C. (2016). XGBoost. Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining - KDD 16. doi: 10.1145/2939672.2939785  
CatBoost:  
Prokhorenkova, Liudmila, Gusev, Gleb, Vorobev, Aleksandr, … Andrey. (2019, January 20). CatBoost: unbiased boosting with categorical features. Retrieved from https://arxiv.org/abs/1706.09516  
numpy:  
Walt, S. V. D., Colbert, S. C., & Varoquaux, G. (2011). The NumPy Array: A Structure for Efficient Numerical Computation. Computing in Science & Engineering, 13(2), 22–30. doi: 10.1109/mcse.2011.37 